In [1]:
pip install optuna 


  Using cached optuna-4.6.0-py3-none-any.whl.metadata (17 kB)
  Using cached alembic-1.17.2-py3-none-any.whl.metadata (7.2 kB)
  Using cached colorlog-6.10.1-py3-none-any.whl.metadata (11 kB)
  Using cached mako-1.3.10-py3-none-any.whl.metadata (2.9 kB)
Using cached optuna-4.6.0-py3-none-any.whl (404 kB)
Using cached alembic-1.17.2-py3-none-any.whl (248 kB)
Using cached colorlog-6.10.1-py3-none-any.whl (11 kB)
Using cached mako-1.3.10-py3-none-any.whl (78 kB)


In [6]:
from sklearn.model_selection import train_test_split
import optuna 
from sklearn.preprocessing import StandardScaler
import pandas as pd 

In [7]:
df=pd.read_csv("C:\\Users\\DELL\\Downloads\\diabetes.csv")

In [8]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [ ]:
## problem there is some missing value which is represented bu 0 in insulin,and mostly some more columns  

In [9]:
df.isnull().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [11]:
import numpy as np 

In [12]:
## so what we can do is repace 0 with mean of that column 
columns_null = ["Glucose","BloodPressure","SkinThickness","Insulin","BMI"]
df[columns_null]=df[columns_null].replace(0,np.nan)
df[columns_null]=df[columns_null].fillna(df[columns_null].mean())


                                  

## split into x and y 
make train and test split 
standard scaler

In [13]:
X=df.drop(columns=["Outcome"],axis=1)
y=df["Outcome"]

In [14]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20,random_state=42)

In [15]:
scale=StandardScaler()
X_train=scale.fit_transform(X_train)
X_test=scale.transform(X_test)

In [16]:
X_train.shape

(614, 8)

In [17]:
X_test.shape

(154, 8)

In [19]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [20]:
from sklearn.model_selection import cross_val_score

In [21]:
def objective_class(trial):## hum ek objective function banayenge jisme hum hyperparameter tuning karenge 
    n_estimators=trial.suggest_int("n_estimators",50,250)
    max_depth=trial.suggest_int("max_depth",2,20)
    ## define our model 
    model=RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        random_state=42
    )
    ## perform 3 cross validation 
    score=cross_val_score(model,X_train,y_train,cv=3,scoring="accuracy").mean()
    return score

In [22]:
## make a study object 
study=optuna.create_study(direction="maximize",sampler=optuna.samplers.TPESampler())
study.optimize(objective_class,n_trials=50)

[I 2025-12-06 01:56:43,023] A new study created in memory with name: no-name-985758b8-0502-4468-a457-351297ab0fe7
[I 2025-12-06 01:56:44,876] Trial 0 finished with value: 0.7670891120675912 and parameters: {'n_estimators': 176, 'max_depth': 7}. Best is trial 0 with value: 0.7670891120675912.
[I 2025-12-06 01:56:47,838] Trial 1 finished with value: 0.7752351347042882 and parameters: {'n_estimators': 239, 'max_depth': 17}. Best is trial 1 with value: 0.7752351347042882.
[I 2025-12-06 01:56:50,025] Trial 2 finished with value: 0.7817551410808226 and parameters: {'n_estimators': 202, 'max_depth': 12}. Best is trial 2 with value: 0.7817551410808226.
[I 2025-12-06 01:56:51,944] Trial 3 finished with value: 0.7670811413996493 and parameters: {'n_estimators': 170, 'max_depth': 19}. Best is trial 2 with value: 0.7817551410808226.
[I 2025-12-06 01:56:53,530] Trial 4 finished with value: 0.7768611509644509 and parameters: {'n_estimators': 87, 'max_depth': 16}. Best is trial 2 with value: 0.781755

In [23]:
study.best_trial.values

[0.7882751474573569]

In [24]:
study.best_trial.params

{'n_estimators': 111, 'max_depth': 13}

In [25]:
best_model=RandomForestClassifier(** study.best_trial.params,random_state=42)
best_model.fit(X_train,y_train)
y_pred=best_model.predict(X_test)
accuracy_score(y_test,y_pred)


0.7532467532467533

In [ ]:
## we have acheived accuracy of 75 percent 

In [30]:
## now lets use some diff sampler 
def objective_class(trial):
    n_estimators=trial.suggest_int("n_estimators",50,250)
    max_depth=trial.suggest_int("max_depth",2,20)
    model=RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        random_state=42
    )
    score=cross_val_score(model,X_train,y_train,cv=3,scoring="accuracy").mean()
    return score

In [31]:
study=optuna.create_study(direction="maximize",sampler=optuna.samplers.RandomSampler())
study.optimize(objective_class,n_trials=50)

[I 2025-12-06 02:21:23,664] A new study created in memory with name: no-name-424dfa14-8b14-404e-9561-8f343c469a2c
[I 2025-12-06 02:21:24,803] Trial 0 finished with value: 0.755706998246453 and parameters: {'n_estimators': 56, 'max_depth': 9}. Best is trial 0 with value: 0.755706998246453.
[I 2025-12-06 02:21:26,495] Trial 1 finished with value: 0.7589430894308943 and parameters: {'n_estimators': 130, 'max_depth': 4}. Best is trial 1 with value: 0.7589430894308943.
[I 2025-12-06 02:21:28,069] Trial 2 finished with value: 0.760569105691057 and parameters: {'n_estimators': 105, 'max_depth': 4}. Best is trial 2 with value: 0.760569105691057.
[I 2025-12-06 02:21:29,267] Trial 3 finished with value: 0.7671050534034752 and parameters: {'n_estimators': 83, 'max_depth': 7}. Best is trial 3 with value: 0.7671050534034752.
[I 2025-12-06 02:21:30,749] Trial 4 finished with value: 0.7752351347042882 and parameters: {'n_estimators': 120, 'max_depth': 18}. Best is trial 4 with value: 0.77523513470428

In [33]:
study.best_trial.values

[0.7866331898613104]

In [34]:
study.best_trial.params

{'n_estimators': 146, 'max_depth': 13}

In [36]:
best_model=RandomForestClassifier(n_estimators=146,max_depth=13,random_state=42)
best_model.fit(X_train,y_train)
y_pred=best_model.predict(X_test)
accuracy_score(y_test,y_pred)

0.7532467532467533